# Aspect based Sentiment Analysis

![](https://images-cdn.newscred.com/Zz0zNjA1NWE3YzNmNDU5NjdlYzA3ZDI1MTdhZDZiMDI4OA==)

In [50]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mamaearth-products-dataset/Mamaearth.csv


In [51]:
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [86]:
DATA_PATH = "/kaggle/input/mamaearth-products-dataset/Mamaearth.csv"


### Importing Modules and Libraries

In [87]:
# import modules and download packages
import nltk
import pandas as pd
nltk.download('words')
nltk.download('punkt')
nltk.download('maxent_ne_chunker')
nltk.download('averaged_perceptron_tagger')
nltk.download('state_union')
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re

[nltk_data] Downloading package words to /usr/share/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package state_union to /usr/share/nltk_data...
[nltk_data]   Package state_union is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [88]:
#importing SentimentIntensityAnalyser from Vader lexicon
nltk_absa = SentimentIntensityAnalyzer()

In [89]:
# MamaEarth dataset
data = pd.read_csv(DATA_PATH)
data

,Product Name,Product Link,Rating,Reviews,MRP,Pack Size,Discount,Key Ingredients,Category,Tokens
0,Tea Tree Facewash for acne & pimples,https://mamaearth.in/product/teatreefacewash,4.8,For baby skin that constantly flaked and dry t...,Rs.249.00,100ml,NaN,['Neem ' 'Aloe Vera ' 'Tea Tree Oil '],['Face Wash'],['tea' 'tree' 'facewash' 'acne' 'pimple']
1,Anti Hair Fall Kit for complete hair care,https://mamaearth.in/product/antihairfallkit,4.8,With changing weather I tend to get itchy and ...,Rs.1199.00,NaN,(25% off),['Bhringraj Oil ' 'Biotin ' 'All hair type Th...,['Hair Kit'],['anti' 'hair' 'fall' 'kit' 'complete' 'hair' ...
2,Argan Hair Mask with Argan Avocado Oil and Mil...,https://mamaearth.in/product/arganhairmask,4.8,It is the best charcoal face mask i have ever ...,Rs.599.00,200ml,NaN,['Argan Oil '],['Hair Oil'],['argan' 'hair' 'mask' 'argan' 'avocado' 'oil'...
3,Ubtan Bamboo Sheet Mask with Turmeric & Saffro...,https://mamaearth.in/product/ubtanbamboosheetm...,5.0,Very good product with good customer service,Rs.149.00,25 g,NaN,['Turmeric'],['NA'],['ubtan' 'bamboo' 'sheet' 'mask' 'turmeric' 's...
4,Niacinamide Bamboo Sheet Mask with Niacinamide...,https://mamaearth.in/product/niacinamidebamboo...,5.0,Firstly I would like to say that I like all th...,Rs.149.00,25 g,NaN,['Niacinamide'],['NA'],['niacinamide' 'bamboo' 'sheet' 'mask' 'niacin...
...,...,...,...,...,...,...,...,...,...,...
261,Argan Hair Mask with Argan Avocado Oil and Mil...,https://mamaearth.in/product/arganhairmask,4.8,It is the best charcoal face mask i have ever ...,Rs.599.00,200ml,NaN,['Argan Oil '],['Hair Oil'],['argan' 'hair' 'mask' 'argan' 'avocado' 'oil'...
262,C3 Face Mask for healthy & glowing skin,https://mamaearth.in/product/buycharcoalfacemask,5.0,Mamaearth body wash for baby is just wow...it ...,Rs.599.00,100ml,NaN,['Charcoal ' 'Coffee ' 'Kaolin Clay ' 'Acne Pr...,['Face Mask'],['c3' 'face' 'mask' 'healthy' 'glow' 'skin']
263,Ubtan Body Wash With Turmeric & Saffron for Gl...,https://mamaearth.in/product/ubtanbodywashwith...,NaN,NaN,Rs.598.00,Pack Of 2,NaN,['Turmeric'],['Body Wash'],['ubtan' 'body' 'wash' 'turmeric' 'saffron' 'g...
264,Vitamin C Bamboo Sheet Mask with Vitamin C & H...,https://mamaearth.in/product/vitamincbambooshe...,5.0,Really Amazing Product Worth it Must try!,Rs.149.00,25 g,NaN,['Vitamin C'],['NA'],['vitamin' 'c' 'bamboo' 'sheet' 'mask' 'vitami...


### Data Cleaning

In [90]:
data.isnull().sum()

Product Name         0
Product Link         0
Rating             112
Reviews            104
MRP                  0
Pack Size           62
Discount           190
Key Ingredients      0
Category             0
Tokens               0
dtype: int64

In [91]:
data = data.drop(columns=['Product Link','MRP','Rating','Discount','Pack Size',])

In [92]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Product Name     266 non-null    object
 1   Reviews          162 non-null    object
 2   Key Ingredients  266 non-null    object
 3   Category         266 non-null    object
 4   Tokens           266 non-null    object
dtypes: object(5)
memory usage: 10.5+ KB


We need not concern ourselves with dropping the null columns as we will be only working with some of the columns.
For our work, we only need to concern ourselves with :-
* Product Name
* Category
* Reviews
* Key Ingredients
* Category
* Tokens

In [93]:
data = data.astype(
    {"Product Name" : "string",
     "Category" : "string",
     "Reviews" : "string",
     "Key Ingredients" : "string",
     "Tokens" : "string"}
)

In [94]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Product Name     266 non-null    string
 1   Reviews          162 non-null    string
 2   Key Ingredients  266 non-null    string
 3   Category         266 non-null    string
 4   Tokens           266 non-null    string
dtypes: string(5)
memory usage: 10.5 KB


In [108]:
data_1 = data.dropna()

In [109]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 162 entries, 0 to 265
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Product Name        162 non-null    string
 1   Reviews             162 non-null    string
 2   Key Ingredients     162 non-null    string
 3   Category            162 non-null    string
 4   Tokens              162 non-null    string
 5   Product_Sentiments  162 non-null    object
dtypes: object(1), string(5)
memory usage: 8.9+ KB


We will be performing simple aspect based sentiment analysis (ASBA)
The aspects we will be looking out for ASBA are taken from :-
* Product Name
* Category
* Key Ingredients
* Category
* Tokens

Each of these elements will be searched for in reviews and the respective sentence's sentiment will be calculated and tabluated.

#### **NLTK VADER LEXICON** 
( link : https://www.nltk.org/_modules/nltk/sentiment/vader.html )

VADER( Valence Aware Dictionary for Sentiment Reasoning) is an NLTK module that provides sentiment scores based on the words used. It is a rule-based sentiment analyzer in which the terms are generally labeled as per their semantic orientation as either positive or negative.

In [110]:
# ASBA function that returns sentiments for eacha nd every row
def get_aspect_sentiments(reviews, aspects, product_name, category, key_ingredients):
    aspect_sentiments = {}
    aspects = list(re.findall("'([^']*)'", aspects))
    ingred = list(re.findall("'([^']*)'", key_ingredients)) 
    for x in ingred:
        aspects.append(x)
    categ = list(re.findall("'([^']*)'", category))
    for x in categ:
        aspects.append(x)
    for x in ingred:
        aspects.append(x)
    aspects.append(product_name)
    aspects.append(" ")

    sentences = nltk.sent_tokenize(reviews)

    for aspect in aspects:
        aspect_sentiments[aspect] = {'Positive': 0, 'Negative': 0, 'Neutral': 0}
        
# Using polarity_scores from analyzer to tabulate the sentiment scores into positive negative and neutral
    for sentence in sentences:
        for aspect in aspects:
            if aspect in sentence.lower():
                sentiment_score = nltk_absa.polarity_scores(sentence)
                if sentiment_score['compound'] >= 0.05:
                    aspect_sentiments[aspect]['Positive'] += 1
                elif sentiment_score['compound'] < -0.05:
                    aspect_sentiments[aspect]['Negative'] += 1
                else:
                    aspect_sentiments[aspect]['Neutral'] += 1

    return aspect_sentiments

In [111]:
# Applying sentiment Analysis
data_1["Product_Sentiments"] = data_1.apply(lambda row : get_aspect_sentiments(row["Reviews"], row["Tokens"], row["Product Name"], row["Category"], row["Key Ingredients"]),axis= 1)

#### Sentiments based on the given aspects

In [112]:
data_1["Product_Sentiments"][247]

{'healthy': {'Positive': 0, 'Negative': 0, 'Neutral': 0},
 'hair': {'Positive': 0, 'Negative': 0, 'Neutral': 0},
 'combo': {'Positive': 0, 'Negative': 0, 'Neutral': 0},
 'shampoo': {'Positive': 0, 'Negative': 0, 'Neutral': 0},
 'conditioner': {'Positive': 0, 'Negative': 0, 'Neutral': 0},
 'Hair Shampoo': {'Positive': 0, 'Negative': 0, 'Neutral': 0},
 'Healthy Hair Combo : Shampoo and Conditioner': {'Positive': 0,
  'Negative': 0,
  'Neutral': 0},
 ' ': {'Positive': 1, 'Negative': 0, 'Neutral': 1}}

#### The review

In [113]:
data_1["Reviews"][247]

'Very smooth and light on skin and provides long lasting hydration. It is also very cooling which i like during summer time. Nimmi'

We put it in each row for every product

In [114]:
data_1

,Product Name,Reviews,Key Ingredients,Category,Tokens,Product_Sentiments
0,Tea Tree Facewash for acne & pimples,For baby skin that constantly flaked and dry t...,['Neem ' 'Aloe Vera ' 'Tea Tree Oil '],['Face Wash'],['tea' 'tree' 'facewash' 'acne' 'pimple'],"{'tea': {'Positive': 0, 'Negative': 0, 'Neutra..."
1,Anti Hair Fall Kit for complete hair care,With changing weather I tend to get itchy and ...,['Bhringraj Oil ' 'Biotin ' 'All hair type Th...,['Hair Kit'],['anti' 'hair' 'fall' 'kit' 'complete' 'hair' ...,"{'anti': {'Positive': 0, 'Negative': 0, 'Neutr..."
2,Argan Hair Mask with Argan Avocado Oil and Mil...,It is the best charcoal face mask i have ever ...,['Argan Oil '],['Hair Oil'],['argan' 'hair' 'mask' 'argan' 'avocado' 'oil'...,"{'argan': {'Positive': 0, 'Negative': 0, 'Neut..."
3,Ubtan Bamboo Sheet Mask with Turmeric & Saffro...,Very good product with good customer service,['Turmeric'],['NA'],['ubtan' 'bamboo' 'sheet' 'mask' 'turmeric' 's...,"{'ubtan': {'Positive': 0, 'Negative': 0, 'Neut..."
4,Niacinamide Bamboo Sheet Mask with Niacinamide...,Firstly I would like to say that I like all th...,['Niacinamide'],['NA'],['niacinamide' 'bamboo' 'sheet' 'mask' 'niacin...,"{'niacinamide': {'Positive': 0, 'Negative': 0,..."
...,...,...,...,...,...,...
260,Mamaearth Under Eye CrÃ¨me with Cucumber & Caf...,Mamaearth onion hair mask is an awesome produc...,['Cucumber Extract '],['NA'],['mamaearth' 'eye' 'crÃ¨me' 'cucumber' 'caffei...,"{'mamaearth': {'Positive': 1, 'Negative': 0, '..."
261,Argan Hair Mask with Argan Avocado Oil and Mil...,It is the best charcoal face mask i have ever ...,['Argan Oil '],['Hair Oil'],['argan' 'hair' 'mask' 'argan' 'avocado' 'oil'...,"{'argan': {'Positive': 0, 'Negative': 0, 'Neut..."
262,C3 Face Mask for healthy & glowing skin,Mamaearth body wash for baby is just wow...it ...,['Charcoal ' 'Coffee ' 'Kaolin Clay ' 'Acne Pr...,['Face Mask'],['c3' 'face' 'mask' 'healthy' 'glow' 'skin'],"{'c3': {'Positive': 0, 'Negative': 0, 'Neutral..."
264,Vitamin C Bamboo Sheet Mask with Vitamin C & H...,Really Amazing Product Worth it Must try!,['Vitamin C'],['NA'],['vitamin' 'c' 'bamboo' 'sheet' 'mask' 'vitami...,"{'vitamin': {'Positive': 0, 'Negative': 0, 'Ne..."


In [115]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 162 entries, 0 to 265
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Product Name        162 non-null    string
 1   Reviews             162 non-null    string
 2   Key Ingredients     162 non-null    string
 3   Category            162 non-null    string
 4   Tokens              162 non-null    string
 5   Product_Sentiments  162 non-null    object
dtypes: object(1), string(5)
memory usage: 12.9+ KB


#### Saving Results

Sucessfully analyzed sentiments for all the considered aspects.
Now you can just download the file from " /kaggle/working/ " to see your results.
Please upvote if you think its good and do give feedback.
Thank you for reading so far !!

P.S. This is my first notebook that I am publishing.

In [116]:
data.to_csv("/kaggle/working/nltk_asba.csv")